In [ ]:
import serial
import time
import csv
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
PORT = "COM3"  # Change to your Arduino port (Linux: "/dev/ttyUSB0", Mac: "/dev/tty.usbserial-XXXX")
BAUDRATE = 115200
FILENAME = "peltier_data.csv"

# Open serial connection
ser = serial.Serial(PORT, BAUDRATE, timeout=1)
time.sleep(2)  # Give Arduino time to reset

# Open CSV file
with open(FILENAME, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Time (s)", "Temperature (C)", "PWM Level"])

    print("Logging data... Press Ctrl+C to stop.")
    try:
        while True:
            line = ser.readline().decode().strip()
            if line:
                print(line)
                if "Experiment Finished!" in line:
                    break  # Stop when Arduino finishes

                # Parse CSV values
                values = line.split(",")
                if len(values) == 3:
                    writer.writerow(values)

    except KeyboardInterrupt:
        print("\nStopped by user.")

# Close serial connection
ser.close()

# --- PLOTTING DATA ---
import pandas as pd

df = pd.read_csv(FILENAME)

# Plot temperature over time for each PWM level
plt.figure(figsize=(10, 5))
for pwm in df["PWM Level"].unique():
    subset = df[df["PWM Level"] == pwm]
    plt.plot(subset["Time (s)"], subset["Temperature (C)"], label=f"PWM {pwm}")

plt.xlabel("Time (s)")
plt.ylabel("Temperature (C)")
plt.title("Temperature vs. Time for Different PWM Levels")
plt.legend()
plt.grid()
plt.show()
